In [ ]:
!pip install -q --upgrade pip
!pip install -q --upgrade jax[cuda11_cudnn805] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q git+https://www.github.com/google/neural-tangents
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

In [ ]:
!pip uninstall jaxlib
!pip install jaxlib==0.4.1

In [ ]:
import os
import neural_tangents as nt
from neural_tangents import stax
from jax import numpy as np
from jax import random
import jax

In [ ]:
# define a grap convolution network
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Aggregate(aggregate_axis=(1, 2), batch_axis=0, channel_axis=3),
    stax.Conv(100, (1,1)), stax.Relu(),
    stax.Aggregate(aggregate_axis=(1, 2), batch_axis=0, channel_axis=3),
    stax.Conv(100, (1,1)), stax.Relu(),
    stax.Aggregate(aggregate_axis=(1, 2), batch_axis=0, channel_axis=3),
    stax.Conv(100, (1,1)), stax.Relu(),
    stax.Aggregate(aggregate_axis=(1, 2), batch_axis=0, channel_axis=3),
    stax.Conv(100, (1,1)), stax.Relu(),
    stax.Aggregate(aggregate_axis=(1, 2), batch_axis=0, channel_axis=3),
    stax.Conv(100, (1,1)), stax.Relu(),
    stax.Aggregate(aggregate_axis=(1, 2), batch_axis=0, channel_axis=3),
    stax.Conv(100, (1,1)), stax.Relu(),
    stax.Aggregate(aggregate_axis=(1, 2), batch_axis=0, channel_axis=3),
    stax.Conv(100, (1,1)), stax.Relu(),
    stax.Aggregate(aggregate_axis=(1, 2), batch_axis=0, channel_axis=3),
    stax.Conv(100, (1,1)), stax.Relu(),
    stax.Aggregate(aggregate_axis=(1, 2), batch_axis=0, channel_axis=3),
    stax.Conv(100, (1,1)), stax.Relu(),
    stax.Aggregate(aggregate_axis=(1, 2), batch_axis=0, channel_axis=3),
    stax.Conv(100, (1,1)), stax.Relu(),
    stax.GlobalSumPool(), 
)

In [ ]:
# calcualte the kernel for the preprocessed data in splits

base_path = f"/content/drive/MyDrive/MasterarbeitData"
data_path = base_path + f"/Preprocessed/MUTAG"
kernel_path = base_path + f"/Kernels/MUTAG/GCNN"

if not os.path.exists(base_path_splits):
    os.makedirs(kernel_path)

graps_node_features = np.save(data_path + f"/graps_node_features.npy")
graph_conv_pattern = np.save(data_path + f"/graph_conv_pattern.npy")

dataset_size = graps_node_features.shape[0]

size = 37
for i in range(5):
  for j in range(5):

    l = lambda x : x*size 
    h = lambda x : (1+x)*size if (not x == r) else dataset_size-1

    x1 = graps_node_features[l(i):h(i),:]
    x2 = graps_node_features[l(j):h(j),:]
    p1 = graph_conv_pattern[l(i):h(i),:]
    p2 = graph_conv_pattern[l(j):h(j),:]

    kernel_matrix = kernel_fn(x1, x2, 'ntk', pattern=(p1, p2))
    np.save(kernel_path + f"/NTK_{h(i)}_{h(j)}", kernel_matrix)
